In [1]:
%matplotlib inline
import re
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
import regex
import pandas as pd
from elasticsearch_dsl import *
from elasticsearch_dsl import query
from elasticsearch import *
import tqdm
from pathlib import Path


In [2]:
article_reqex = r'( *Art\. )(\d+)(\.((?! *Art)([\s\S]))*)'
pattern = re.compile(article_reqex)
article_directory = 'artykuly'


In [3]:


path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r',encoding='utf-8') as file:
        content = file.read()
        file_name = Path(file.name).stem
#         print(file_name)
        for m in re.finditer(pattern, content):
#             print(m.group(1), '*', m.group(2),'*', m.group(3))
            article_filename =f'{file_name}_{m.group(2)}'
            with open(os.path.join(article_directory, article_filename), 'w',encoding='utf-8') as article_file:
                article_file.write(m.group(0))        


In [4]:
# df = pd.read_csv('questions.csv',names=['passage_id', 'question', 'passage'])

# for filename in os.listdir('artykuly_wybrane'):
#     with open(os.path.join('artykuly_wybrane', filename), 'r',encoding='utf-8') as file: # open in readonly mode
#         content = file.read()
#         file_name = Path(file.name).stem
#         tmp = {'passage_id': file_name, 'question': '', 'passage': content}
#         df = df.append(tmp, ignore_index = True)

# df.to_csv('questions.csv', index=False,header=None)

In [5]:
df = pd.read_csv('questions1.csv',names=['passage_id', 'question', 'passage'])
df

,passage_id,question,passage
0,2000_1319_9,Kto wydaje decyzję o uznaniu praktyki za ogran...,Art. 9.\nP...
1,2004_574_51,Kto powołuje okręgowych inspektorów rybołówstw...,Art. 51.\n...
2,2001_21_24,Czym jest park krajobrazowy?,Art. 24. 1. Park krajobrazowy jest obszarem ch...
3,1994_602_4,W jakim momencie powstaje obowiązek podatkowy?,Art. 4.\nObowi...
4,1996_465_134,W jakim momencie powstaje spółka komandytowo-a...,Art. 134.\...
5,2004_879_14,Ile przynajmniej godzin odpoczynku przysługuje...,Art. 14.\n...
6,2003_1750_112,Czy skazanie prawomocnym wyrokiem sądowym może...,Art. 112.\...
7,2001_298_39,Kogo zwalnia ze służby szef BOR?,Art. 39.\nFunk...
8,1995_368_37,Czy skarga bez wskazania określonego naruszeni...,Art. 37.\nSkar...
9,1997_944_10,Podczas jakich wydarzeń przełożony jest zobowi...,Art. 10.\n 1. ...


# Elastic Search

In [6]:

es = Elasticsearch("http://localhost:9200")
resp = es.info()
resp


{'name': 'x-dell',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'iBI8GYH1TzOb1ag0UsGzTg',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [7]:
connections.add_connection('python_client', es)
connections.create_connection(hosts=['localhost'])

print(es.cat.plugins(params={'v': '', 'h': 'name,component,version,description'}))



name   component           version description
x-dell analysis-morfologik 7.10.1  Morfologik Polish Lemmatizer plugin for Elasticsearch



In [8]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)


/home/x/anaconda3/envs/pjn1/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [9]:
from haystack.utils import launch_es
launch_es()


8a217c7c8ebcd4d74567bcb9e6c3cbe279981266ddc810fc780fd22ecb7c6ff4


docker: Error response from daemon: driver failed programming external connectivity on endpoint strange_benz (15dd1d21b519e2ec133a83569644cc14d6193d9f1989259aca01f3563e3d1b64): Error starting userland proxy: listen tcp4 0.0.0.0:9200: bind: address already in use.
WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [10]:
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="artykul")


In [11]:
artykuly = []
for filename in os.listdir(article_directory):
    with open(os.path.join(article_directory, filename), 'r',encoding='utf-8') as file:
        match = re.match(pattern,file.read())
        
        artykul = {
            'content': match.group(3),
            'meta': {
                'name': Path(file.name).stem,
                'artykul_id' : match.group(2)
            }
        }
        artykuly.append(artykul)
        
# print(artykuly[1])
document_store.write_documents(artykuly)


In [12]:
df.iloc[3]['question']

'W jakim momencie powstaje obowiązek podatkowy?'

In [31]:
class color: #kolorki
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


In [87]:
from haystack.nodes import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)
number_of_chosen_articles = 30

top_k=1


def print_aritcles(retriever,df,number_of_chosen_articles,top_k=1,falseOnly = False):
    number_of_found_correct=0
    for question_id in range(number_of_chosen_articles):

        candidate_documents = retriever.retrieve(
            query=df.iloc[question_id]['question'],
            top_k=top_k
        )
        
        for i in range(top_k):
            found_correct = df.iloc[question_id]['passage_id'] == candidate_documents[i].meta['name']
            if found_correct:
                number_of_found_correct+=1
                
            
            if falseOnly and found_correct:
                continue
            
            print(f"{color.RED}{df.iloc[question_id]['question']}{color.END}")

            print(f"{color.PURPLE} AnswerId:{i} Score:{candidate_documents[i].score} plik:{candidate_documents[i].meta['name']} artykul_id:{candidate_documents[i].meta['artykul_id']} {color.END}")
            if top_k<5:
                print(f"Znaleziony artykul to ten na podstawie którego układano pytanie:{color.PURPLE}{found_correct}{color.END}")


            print(candidate_documents[i].content)

        print(f'{color.BLUE}$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$${color.BLUE}')
    return number_of_found_correct


In [88]:
res =print_aritcles(retriever,df,number_of_chosen_articles = 30,top_k=1)


Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
 AnswerId:0 Score:0.998199619060783 plik:2000_1319_9 artykul_id:9 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
Prezes Urzędu Ochrony Konkurencji i Konsumentów, zwany dalej "Prezesem Urzędu",
wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję i nakazującą
zaniechanie jej stosowania, jeżeli stwierdzi naruszenie zakazu określonego w
art. 5, w zakresie niewyłączonym na podstawie art. 6 i 7, lub naruszenie art.
8.
                                       

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Kto powołuje okręgowych inspektorów rybołówstwa morskiego?
 AnswerId:0 Score:0.9967940700668187 plik:2001_1441_62 artykul_id:62 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Okręgowi inspektorzy rybołówstwa morskiego podlegają ministrowi
  właściwemu do spraw rolnictwa.
2. Okręgowych inspektorów rybołówstwa morskiego powołuje i odwo

Kto wybiera likwidatora spółdzielni?
 AnswerId:0 Score:0.9457262645661288 plik:2003_1939_81 artykul_id:81 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
 § 1. Likwidatora spółdzielni wybiera walne zgromadzenie, a w wypadku określonym
     w art. 75 - sąd. Likwidatorem może być osoba, która posiada kwalifikacje
     określone przez Radę Spółdzielczą. Likwidatorem może być także osoba
     prawna.
 § 2. Likwidatorem nie może być osoba, która wchodziła w skład zarządu
     spółdzielni w ciągu roku przed dniem otwarcia jej likwidacji.
 § 3. Umowę z likwidatorem o wykonanie czynności likwidacyjnych zawiera rada. W
     wypadku, gdy zwołanie rady napotyka poważne trudności, umowę z
     likwidatorem zawiera związek, działając w imieniu spółdzielni.
 § 4. Likwidator może być odwołany w każdej chwili przez organ, który go
     wyznaczył. Likwidatora może również odwołać z ważnych przyczyn związek
     rewizyjny, w którym spółdzielnia jest zrzeszona.
 § 5. Organ lub zwi

In [81]:
number_of_found_correct

0

In [75]:
print_aritcles(retriever,df,number_of_chosen_articles = 30,top_k=1,falseOnly=True)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Kto powołuje okręgowych inspektorów rybołówstwa morskiego?
 AnswerId:0 Score:0.9967940700668187 plik:2001_1441_62 artykul_id:62 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Okręgowi inspektorzy rybołówstwa morskiego podlegają ministrowi
  właściwemu do spraw rolnictwa.
2. Okręgowych inspektorów rybołówstwa morskiego powołuje i odwołuje
  minister właściwy do spraw rolnictwa za zgodą właściwego miejscowo
  wojewody. Zastępców okręgowych inspektorów rybołówstwa morskiego
  powołuje i odwołuje minister właściwy do spraw rolnictwa na wniosek
  okręgowych inspektorów rybołówstwa morskiego.
3. Minister właściwy do spraw rolnictwa określi, w drodze rozporządzenia,
  siedziby i terytorialny zakres działania okręgowych inspektorów
  rybołówstwa morskiego, mając na względzie zwiększenie efektywności
  działania tych organów.
4. Okręgowi inspektorzy rybołówstwa morskiego wykonują swoje zadani

In [90]:
res1 = print_aritcles(retriever,df,number_of_chosen_articles = 30,top_k=3)

Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
 AnswerId:0 Score:0.998199619060783 plik:2000_1319_9 artykul_id:9 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
Prezes Urzędu Ochrony Konkurencji i Konsumentów, zwany dalej "Prezesem Urzędu",
wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję i nakazującą
zaniechanie jej stosowania, jeżeli stwierdzi naruszenie zakazu określonego w
art. 5, w zakresie niewyłączonym na podstawie art. 6 i 7, lub naruszenie art.
8.
                                       

Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
 AnswerId:1 Score:0.9965674058749963 plik:2000_1319_10 artykul_id:10 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Nie wydaje się decyzji, o której mowa w art. 9, jeżeli zachowanie rynkowe
  przedsiębiorcy albo związku przedsiębiorców przestało naruszać zakazy
  określone w art. 5 lub w art. 8, w szczególności z powodu trwałego
  zmniejsze

Na czyj wniosek osoba małoletnia może być zwolniona przez policję?
 AnswerId:0 Score:0.990566887153601 plik:2003_2275_404 artykul_id:404 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
. Osoba małoletnia
doprowadzona do izby wytrzeźwień, placówki lub jednostki Policji może być
zwolniona, niezwłocznie po udzieleniu jej niezbędnych świadczeń zdrowotnych, na
pisemny wniosek rodziców lub opiekunów.”;
6)   art. 41 i
42 otrzymują brzmienie:
„
Na czyj wniosek osoba małoletnia może być zwolniona przez policję?
 AnswerId:1 Score:0.9278457960097315 plik:1997_555_186 artykul_id:186 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
 § 1. Jeżeli osoba uprawniona do odmowy złożenia zeznania albo zwolniona 
     od tego na podstawie art. 185 oświadczy nie później niż przed rozpoczęciem
     pierwszego zeznania na rozprawie przed sądem pierwszej instancji,
     że chce z tego prawa skorzystać, poprzednio złożone zeznanie 
     tej osoby nie może ani służyć 

In [100]:
print(f'Odnalezione w top1:{res}/30 Odnalezione w top3:{res1}/30')

Odnalezione w top1:22/30 Odnalezione w top3:27/30


In [94]:
#1

relevant_and_retrieved =  res + 6
retrieved = 30
relevant = 30
precision = relevant_and_retrieved/retrieved
recall = relevant_and_retrieved/relevant

print(f'precision:{precision}  recall:{recall}')

precision:0.9333333333333333  recall:0.9333333333333333


In [96]:
#3
relevant_and_retrieved =  res1
retrieved = 90
relevant = 90
precision = relevant_and_retrieved/retrieved
recall = relevant_and_retrieved/relevant

print(f'precision:{precision}  recall:{recall}')


precision:0.3  recall:0.3


10. Answer the following questions:
    1. Which of the document stores performs better? Take into account the different metrics enumerated in the previous
       point.
W przypadku ElasticSearch i top1 znaleziono 22 spośród 30 początkowych artykułów. 6 pozostałych odpowiedzi miało sens. Często były to bardzo zbliżone treścią artykuł do tych z których układałem pytania, ale znalazły się one w innej ustawie. Dwie sensu nie miały (Czy Ochotnicze Hufce Pracy zajmują się młodzieżą zaniedbaną wychowawczo?, Jak nazywa się najwyższa samorządowa organizacja ruchu spółdzielczego?) W odpowiedziach podano definicję Hufców Pracy i NIK. W top3 znalazło się już 27 spośród 30 początkowych artykułów. 
       
    2. Which of the document stores is faster?
    
    Dense document retrieval zajmuje bardzo dużo czasu podczas indeksowania i wymaga użycia karty graficznej żeby przyspieszyć ten czas. Wynika to z tego, że 
    Zdecydowanie ElasticSearch radzi sobie pod tym względem lepiej. 
    
    
    3. Try to determine the other pros and cons of using sparse and dense document retrieval models.
    
Według dokumentacji "Dense methods perform indexing by processing all the documents through a neural network and storing the resulting vectors. This is a much more expensive operation than the creation of the inverted-index in sparse methods and will require significant computational power and time"
   
